In [1]:
import asana
from datetime import date, datetime, timezone
import argparse
import sys, os
import json
import xml.dom.minidom as xr
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta




In [2]:
# dates in NAV changed format at some point: try different format to read dates from NAV xml
def get_date(date_str):
    date_formats = ["%d/%m/%y", "%y-%m-%d", "%Y-%m-%d","%Y/%m/%d","%m/%d/%y"]
    for date_format in date_formats:
        try:
            return datetime.strptime( date_str, date_format )
        except:
            pass

In [3]:
# Parse xml
doc = xr.parse("../data/2012_6692.xml")
# Get some attributes: Job Order number and delivery date (to the client), expected processing init and end
JOnumber = doc.getElementsByTagName("JobOrderCode")[0].firstChild.nodeValue
delivery_code = doc.getElementsByTagName("Delivery")[0].getElementsByTagName("Code")[0].firstChild.nodeValue
delivery_date = doc.getElementsByTagName("Delivery")[0].getElementsByTagName("Date")[0].firstChild.nodeValue
# All dates are strings, change to datetime format
delivery_date = get_date(delivery_date[:10])
expected_init = get_date(doc.getElementsByTagName("DeliveryExpectedInit")[0].firstChild.nodeValue)
expected_end = get_date(doc.getElementsByTagName("DeliveryExpectedProcessingEnd")[0].firstChild.nodeValue)

In [4]:
# Personal access token
personal_access_token = '1/1202212364519907:bfcacb7b5b0fa7ed5773e96e270cf977'

# Construct an Asana client
client = asana.Client.access_token(personal_access_token)

# Get your user info
me = client.users.me()


In [5]:
workspace_gid = "186860237544930" # TRE Altamira
team_gid = "735937443511725" # Geospatial Solutions Italy
proc_gid = "1167558511333185" # Processing
template_gid = "1203433335473715" # benji_multiteam
projects_overview_gid = "1202122023599837" # GS master project

In [6]:
users_gs = pd.DataFrame(client.users.get_users_for_team(team_gid))
users_proc = pd.DataFrame(client.users.get_users_for_team(proc_gid))




In [9]:
# Duplicate the template project
project_title = "BENJI TEST [JO%s-%s]" % (JOnumber, delivery_code)
dup_project = client.projects.duplicate_project(template_gid, {
        "include": [
            "notes",
            "task_notes",
            "task_subtasks",
            "task_attachments",
            "task_dates",
            "task_dependencies",
            "task_followers",
            "task_tags",
        ],
        "name": project_title,
        "schedule_dates": {
            "start_on": expected_init.strftime("%Y-%m-%d"),
            "should_skip_weekends": False
        },
    })

project_gid = dup_project["new_project"]["gid"]

c:\Users\saadia\Documents\0_WORK\ASANA_DEVELOPMENT\venv\lib\site-packages\asana\client.py:157: UserWarning: This request is affected by the "new_project_templates" deprecation. Please visit this url for more info: https://forum.asana.com/t/a-new-api-for-project-templates/156432
Adding "new_project_templates" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)


In [10]:
# Assign tasks to proper team members
assignee_gs = users_gs[users_gs["name"] == "Benjamin Saadia"]["gid"].values[0] # Placeholder assignee for GS
assignee_proc = assignee_gs # Placeholder assignee for proc

# The tasks that need to be assigned to Processing. In the future, consider using the Team field
proc_task_names = ["Data Processing", "Processing completed", "Data Finalization", "Deliverables Generation", "Update 1- Data Processing", "Update 1- Processing completed", "Update 1-  Data Finalization", "Update 1- Deliverables Generation"]

project_tasks = pd.DataFrame(client.tasks.get_tasks_for_project(project_gid, {'opt_fields': ['gid', 'name']}))
# project_tasks
for task_gid, task_name in zip(project_tasks["gid"], project_tasks["name"]):
    if task_name in proc_task_names:
        client.tasks.update_task(task_gid, {'assignee': assignee_proc})
    else:
        client.tasks.update_task(task_gid, {'assignee': assignee_gs, })
        client.tasks.add_project_for_task(task_gid, {'project': projects_overview_gid})


c:\Users\saadia\Documents\0_WORK\ASANA_DEVELOPMENT\venv\lib\site-packages\asana\client.py:157: UserWarning: This request is affected by the "new_user_task_lists" deprecation. Please visit this url for more info: https://forum.asana.com/t/update-on-our-planned-api-changes-to-user-task-lists-a-k-a-my-tasks/103828
Adding "new_user_task_lists" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)
c:\Users\saadia\Documents\0_WORK\ASANA_DEVELOPMENT\venv\lib\site-packages\asana\client.py:157: UserWarning: This request is affected by the "new_project_templates" deprecation. Please visit this url for more info: https://forum.asana.com/t/a-new-api-for-project-templates/156432
Adding "new_project_templates" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)


In [11]:
# Create number of sections based on n_updates
n_updates = 2
section_data = list(client.sections.get_sections_for_project(project_gid=project_gid)) # Get section data for the project
df_section = pd.DataFrame(section_data) # Convert to df
update_gid = df_section[df_section["name"] == "Update 1"]["gid"].values[0] # GID of Update 1 which will be used as a template Section for the rest of the updates
update_body = client.sections.get_section(update_gid)
update_tasks = list(client.tasks.get_tasks_for_section(section_gid=update_gid))
df_update_tasks = pd.DataFrame(update_tasks)
df_update_tasks["name"] = df_update_tasks["name"].str.replace("Update 1- ", "")



NotFoundError: Not Found: project: Not a recognized ID: 1203448067777256

In [12]:
# Iterate through each section in reverse order
for n in reversed(range(2, n_updates+2)):
    new_section = client.sections.create_section_for_project(project_gid, {'name': 'Update %s' % (str(n)), 'insert_after': update_gid})
    for task_gid, task_name in zip(df_update_tasks["gid"],df_update_tasks["name"]):
        dup_task = client.tasks.duplicate_task(task_gid, {'name': 'Update %s- %s' % (str(n), task_name), "include": ["dates", "assignee", "projects"]})
        try:
           start_on = datetime.strptime(client.tasks.get_task(dup_task["new_task"]["gid"])["start_on"], "%Y-%m-%d") + relativedelta(months=n-1)
           start_on = start_on.strftime("%Y-%m-%d")
        except TypeError:
            start_on = ""
        try:
            due_on = datetime.strptime(client.tasks.get_task(dup_task["new_task"]["gid"])["due_on"], "%Y-%m-%d") + relativedelta(months=n-1)
            due_on = due_on.strftime("%Y-%m-%d")
        except TypeError:
            due_on = ""
        client.tasks.update_task(dup_task["new_task"]["gid"], {'start_on': start_on, 'due_on': due_on})
        client.tasks.add_project_for_task(dup_task["new_task"]["gid"], {'project': project_gid, 'section': new_section["gid"]}, opt_pretty=True)

c:\Users\saadia\Documents\0_WORK\ASANA_DEVELOPMENT\venv\lib\site-packages\asana\client.py:157: UserWarning: This request is affected by the "new_project_templates" deprecation. Please visit this url for more info: https://forum.asana.com/t/a-new-api-for-project-templates/156432
Adding "new_project_templates" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)
c:\Users\saadia\Documents\0_WORK\ASANA_DEVELOPMENT\venv\lib\site-packages\asana\client.py:157: UserWarning: This request is affected by the "new_user_task_lists" deprecation. Please visit this url for more info: https://forum.asana.com/t/update-on-our-planned-api-changes-to-user-task-lists-a-k-a-my-tasks/103828
Adding "new_user_task_lists" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)
